<div align="center">
    <img src="images/logo.png" width="250" height="200">
</div>

<div align="center">
    To download the previously cleaned dataset click <a href="https://drive.google.com/file/d/1mYJws7uPjbPqPszUQHpf4PfrHSEjAVIu/view?usp=sharing" target="__blank">here</a>.
</div>

In [ ]:
# import packages
import numpy as np
import pandas as pd

# set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

In [ ]:
# read the data
df = pd.read_pickle('clean_complaints.pkl')
print("Shape: ", df.shape)
df.head(3)

In [ ]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False) # set simple_zipcode=False to use rich info database

def complaints_per_zipcode(passed_df):
    complaints = pd.DataFrame({'ZIP code': [], 'Number of Complaints': [], 'lat': [], 'lng': [], 'polygon': []})
    
    # iterate aall unique zips
    for zip in passed_df['ZIP code'].unique():
        zip_code = str(zip)
        number_of_complaints = passed_df[passed_df['ZIP code'] == zip].shape[0]
        
        # search for zip code's latitude, longtute, and polygons
        zipcode = search.by_zipcode(zip_code)
        zipcode_dict = zipcode.to_dict()
        needed_info = {key: zipcode_dict[key] for key in ['lat', 'lng', 'polygon']}
        
        lat = needed_info['lat']
        lng = needed_info['lng']
        polygon = needed_info['polygon']
        
        new_row = {'ZIP code': zip_code, 'Number of Complaints': number_of_complaints,
                  'lat': lat, 'lng': lng, 'polygon': polygon}
        complaints = complaints.append(new_row, ignore_index=True)
        
    return complaints

In [ ]:
complaints = complaints_per_zipcode(df)

complaints.set_index('ZIP code', inplace=True)
complaints.sort_values(by=['Number of Complaints'], inplace=True, ascending=False)

complaints.head()

In [ ]:
print("There are", complaints['Number of Complaints'].sum(), "number of complaints in the new dataset (same as the initial dataset)")
print("There are", complaints.shape[0], "number of unique ZIP codes")

In [ ]:
# REQUIRES a geoJSON file
# NEEDS saved complaints dataframe as json
# import folium

# complaints_map = folium.Map(location=[39.50, -98.35], zoom_start=4)

# complaints_map.choropleth(
#     geo_data=complaints_no_index['polygon'].tolist()[0:2],
#     name="choropleth",
#     data=complaints['Number of Complaints'],
#     columns=[""],
#     key_on="feature.id",
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name="Unemployment Rate (%)",
# ).add_to(complaints_map)

# complaints_map

In [ ]:
complaints.to_json('complaints.json')

In [ ]:
complaints_no_index = complaints.reset_index()
complaints_no_index = complaints_no_index[['lat', 'lng', 'polygon']]
complaints_no

In [ ]:
complaints_no_index['polygon'].tolist()[0:2]